In [598]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn import preprocessing
import math
from datetime import datetime

In [760]:
data_inventory = pd.read_csv(r"C:\Users\20193727\Downloads\Data\Data\inventory.csv")
data_products = pd.read_csv(r"C:\Users\20193727\Downloads\Data\Data\products.csv")
data_promotions = pd.read_csv(r"C:\Users\20193727\Downloads\Data\Data\promotions.csv")
data_transactions = pd.read_csv(r"C:\Users\20193727\Downloads\Data\Data\transactions.csv")
data_simulprom = pd.read_csv(r"C:\Users\20193727\Downloads\transactions_months_simulprom_subst.csv")

In [725]:
data_holiday = pd.read_excel(r"C:\Users\20193727\Downloads\feestdagen_nl_b_2018.xlsx")

In [898]:
data_simulprom


,Unnamed: 0,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic,date_time,month,simul_promotions,substitute_in_promo,holiday
0,0,1/1/2018,9:07:00,0.0,NaN,snack,s_5,Eierkoeken,4st,1.79,1.79,0.0,0.0,2018-01-01 09:07:00,1,0,0,0
1,1,1/1/2018,9:07:00,0.0,NaN,meat,m_16,Sappige gelders rookworst,250g,1.08,1.08,0.0,0.0,2018-01-01 09:07:00,1,4,0,0
2,2,1/1/2018,9:07:00,0.0,NaN,meat,m_18,Varkensbraadworst,4st,2.49,1.99,NaN,0.0,2018-01-01 09:07:00,1,4,0,0
3,3,1/1/2018,9:07:00,0.0,NaN,meat,m_6,Half om half gehakt,500g,2.69,2.42,0.0,0.0,2018-01-01 09:07:00,1,4,0,0
4,4,1/1/2018,9:07:00,0.0,NaN,bread,b_33,Biologisch bruin heel,1st,2.12,2.12,1.0,0.0,2018-01-01 09:07:00,1,0,0,0
5,5,1/1/2018,9:07:00,0.0,NaN,meat,m_9,Mini biefstukjes,180g,2.39,2.39,0.0,0.0,2018-01-01 09:07:00,1,4,0,0
6,6,1/1/2018,9:07:00,0.0,NaN,snack,s_1,Vanille muffins (vers),1st,3.07,3.07,0.0,0.0,2018-01-01 09:07:00,1,0,0,0
7,7,1/1/2018,9:07:00,0.0,NaN,meat,m_13,Mini gourmet kipsatespiesjes,200g,2.49,2.49,0.0,0.0,2018-01-01 09:07:00,1,4,0,0
8,8,1/1/2018,9:07:00,0.0,NaN,vegetable,v_21,Boerenkool gesneden,300g,1.29,1.29,0.0,0.0,2018-01-01 09:07:00,1,0,0,0
9,9,1/1/2018,9:07:00,0.0,NaN,fruit,f_7,Pitloze rode druiven,500g,2.19,2.19,0.0,0.0,2018-01-01 09:07:00,1,0,0,0


In [897]:
datetime.strptime(data_simulprom['date_time'].unique()[30], '%Y-%m-%d')

ValueError: unconverted data remains:  15:17:34

In [876]:
def parse_holiday(data_hol):
    data_hol = data_hol[data_holiday['Land'] == 'NL']
    data_hol['day'] = 0
    data_hol['day2'] = 0
    data_hol['day_of_year'] = 0
    
    for i in range(len(data_hol.index)):
        data_hol['day'][i] = data_hol['Datum'][i].strftime('%Y-%m-%d')
        data_hol['day'][i] = datetime.strptime(data_hol['day'][i], '%Y-%m-%d').strftime('%d-%m-%Y')
        data_hol['day2'][i] = datetime.strptime(data_hol['day'][i], '%d-%m-%Y').strftime('%d/%m/%Y')
        data_hol['day_of_year'][i] = data_hol['Datum'][i].timetuple().tm_yday
        
    return data_hol
    
    

In [877]:
data_holiday = parse_holiday(data_holiday)
data_holiday


C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while w

,Datum,Omschrijving,Land,day,day2,day_of_year
0,2018-01-01,Nieuwjaarsdag,NL,01-01-2018,01/01/2018,1
1,2018-03-30,Goede vrijdag,NL,30-03-2018,30/03/2018,89
2,2018-04-01,1e Paasdag,NL,01-04-2018,01/04/2018,91
3,2018-04-02,2e Paasdag,NL,02-04-2018,02/04/2018,92
4,2018-04-27,Koningsdag,NL,27-04-2018,27/04/2018,117
5,2018-05-05,Bevrijdingsdag,NL,05-05-2018,05/05/2018,125
6,2015-05-10,Hemelvaartsdag,NL,10-05-2015,10/05/2015,130
7,2018-05-20,1e Pinksterdag,NL,20-05-2018,20/05/2018,140
8,2018-05-21,2e Pinksterdag,NL,21-05-2018,21/05/2018,141
9,2018-12-25,1e Kerstdag,NL,25-12-2018,25/12/2018,359


In [907]:
len(data_simulprom['day'].unique())

364

In [955]:
def add_holidays(data_prom, data_holiday):
    data_new = data_prom
    data_prom['holiday'] = 0
    #nr_days = len(data_prom['day'].unique())+2
    
    for day in range(1, len(data_prom['day'].unique())+1):
        unique_days = data_prom['day'].unique()
        date = unique_days[day-1]
        
        if day in data_holiday['day_of_year']:
            idx_data = data_prom[data_new['day'] == date]
            idx = idx_data.index
            data_prom.loc[idx, 'holiday'] = 1
            
    return data_prom
    
    

In [923]:
def daily_sales(total_data, product):
    
    product_data = total_data[total_data['description']==product]
    category = product_data['category'].iloc[0]
    
    size_nan = product_data['size'].unique()
    size_nan = [x for x in size_nan if x == x]
    
    new_df = pd.DataFrame()
    total_days_size = len(total_data['day'].unique()) * len(size_nan)
    rows = [x for x in range(total_days_size)]
    new_df['idx'] = rows
    
    new_df['week'] = 0
    new_df['daily_sales'] = 0
    new_df['size'] = 0
    new_df['purchase_price'] = 0
    new_df['month'] = 0
    new_df['simul_promotions'] = 0
    new_df['substitute_promotions'] = 0
    new_df['day'] = 0
    new_df['holiday'] = 0
    new_df['category'] = category
    new_df['product'] = product
    
    week = 1
    index = 0
    day_nr = 0
    
    for day in total_data['day'].unique():
        daily_sales = product_data[product_data['day'] == day]
        data_day = total_data[total_data['day'] == day]
        
        size_int = 0
        for size in size_nan:
            relevant_sales = daily_sales[daily_sales['size'] == size]
            indexors = relevant_sales.index
            
            new_df.loc[index, 'daily_sales'] = len(relevant_sales)
            new_df.loc[index, 'size'] = size_int
            new_df.loc[index, 'week'] = week
            new_df.loc[index, 'day'] = day 
            new_df.loc[index, 'month'] = total_data[total_data['day'] == day]['month'].iloc[0]
            
            try:
                new_df.loc[index, 'simul_promotions'] = product_data[product_data['day'] == day]['simul_promotions'].iloc[0]
            except:
                new_df.loc[index, 'simul_promotions'] = 0
            
            try: 
                new_df.loc[index, 'substitute_promotions'] = relevant_sales['substitute_in_promo'].max()
            except:
                new_df.loc[index, 'substitute_promotions'] = 0
            
            try:
                new_df.loc[index, 'purchase_price'] = relevant_sales['purchase_price'].iloc[0]
            except: 
                new_df.loc[index, 'purchase_price'] = product_data['purchase_price'].max()
                
            try:
                new_df.loc[index, 'holiday'] = data_day['holiday'].iloc[0]
            except:
                new_df.loc[index, 'holiday'] = 0
            
            index += 1
            size_int += 1
        
        day_nr += 1
        week = (day_nr // 7) + 1
        
    
    return new_df

In [929]:
def daily_to_restock(data_sales, data_restock):
    

    month_list = []
    price_list = []
    size_list = []
    sales_list = []
    simulprom_list = []
    days_inbetween_list = []
    subst_list = []
    holiday_list = []
    category = data_sales['category'].iloc[0]
    product = data_sales['product'].iloc[0]
    
    
    for size in data_sales['size'].unique():
        rel_data = data_sales[data_sales['size'] == size]
        rel_data.reset_index()
        day_before = 0
        
        for day in data_inventory['day'].unique()[1:]:
            rel_sales = rel_data[day_before : day]
            
            sales = sum(rel_sales['daily_sales'])
            month = rel_sales['month'].iloc[0]
            price = rel_sales['purchase_price'].iloc[0]
            simulprom = rel_sales['simul_promotions'].iloc[0]
            subst = rel_sales['substitute_promotions'].max()
            holiday = rel_sales['holiday'].max()
            days_inbetween = (day - day_before)
            
            holiday_list.append(holiday)
            sales_list.append(sales)
            size_list.append(size)
            month_list.append(month)
            price_list.append(price)
            simulprom_list.append(simulprom)
            subst_list.append(subst)
            days_inbetween_list.append(days_inbetween)
            
            day_before = day
            
    raw_data = {'month':month_list,  'size':size_list, 'price':price_list, 'sales':sales_list,
                'simul_promotions': simulprom_list, 'days_inbetween': days_inbetween_list, 'substitute_promotions': subst_list,
                'holiday':holiday_list}
    new_data = pd.DataFrame(raw_data)
    new_data['category'] = category
    new_data['product'] = product
    return new_data
    
    

In [962]:
def forest_variables(total_data):
    prices = total_data['price']
    sales = total_data['sales']
    sizes = total_data['size']
    months = total_data['month']
    simulprom = total_data['simul_promotions']
    subst_prom = total_data['substitute_promotions']
    days_inbetween = total_data['days_inbetween']
    holidays = total_data['holiday']
    
    zipped = zip(prices, sizes, days_inbetween, months, subst_prom, simulprom, holidays)
    x_variables = [list(a) for a in zipped]
    y_variables = sales
    
    return x_variables, y_variables

In [866]:
def RMSE(bos, X_test, y_test):
    predictions = bos.predict(X_test)
    rounded_predictions = [round(num) for num in predictions]
    SE = 0 
    
    for i in range(len(y_test)):
        squared_error = ((rounded_predictions[i] - y_test.iloc[i])**2)
        SE += squared_error
        
    MSE = SE/ len(y_test)
    MSE = int(MSE)
    
    RMSE = math.sqrt(MSE)
    
    return RMSE
    

In [798]:
def Accuracy(bos, X_test, y_test):
    predictions = bos.predict(X_test)
    rounded_predictions = [round(num) for num in predictions]
    acc = 0
    for i in range(len(y_test)):
        if rounded_predictions[i] == y_test.iloc[i]:
            acc += 1
            
    return acc/len(predictions)

## run preprocessing

In [956]:
data_simulprom_hol = add_holidays(data_simulprom, data_holiday)

In [957]:
data_simulprom_hol[data_simulprom_hol['holiday'] == 1]['day'].unique()

array(['1/1/2018', '2/1/2018', '3/1/2018', '4/1/2018', '5/1/2018',
       '6/1/2018', '7/1/2018', '8/1/2018', '9/1/2018', '10/1/2018',
       '11/1/2018'], dtype=object)

In [958]:
data_daily = daily_sales(data_simulprom_hol, 'Rundergehakt')

In [959]:
restocked_data = daily_to_restock(data_daily, data_inventory)

In [961]:
#restocked_data[restocked_data['holiday'] == 1]

In [559]:
#X_data, y_data = forest_variables(restocked_data)

In [560]:
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.25)

In [976]:
train_df, test_df = train_test_split(restocked_data, test_size = 0.25, stratify = restocked_data[['size', 'month', 'days_inbetween']])

In [977]:
X_train, y_train = forest_variables(train_df)
X_test, y_test = forest_variables(test_df)

## run randomforest

In [965]:
#bos_clf = RandomForestClassifier(n_estimators = 64,
# min_samples_split = 10,
# min_samples_leaf = 2,
# max_features = 'sqrt',
# max_depth = 50,
# bootstrap = True)
#bos_clf.fit(X_train, y_train)


In [978]:
bos_reg = RandomForestRegressor(n_estimators = 64,
         min_samples_split = 10,
         min_samples_leaf = 2,
         max_features = 'sqrt',
         max_depth = 50,
         bootstrap = True)
bos_reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=64, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [813]:
#boom_clf = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
#            max_features=None, max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
#            splitter='best')
#boom_clf.fit(X_train, y_train)

In [967]:
Accuracy(bos_reg, X_test, y_test)

0.057692307692307696

In [979]:
RMSE(bos_reg, X_test, y_test)

5.291502622129181

'2/1/2018'

## parameter_tuning

In [573]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn import tree

In [574]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 64, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [64, 279, 494, 709, 924, 1139, 1354, 1569, 1784, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
#rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 25, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
def tree_parameters(X, y):
    X = np.array(X)
    
    std_slc = StandardScaler()
    pca = decomposition.PCA()
    dec_tree = DecisionTreeClassifier()
    
    pipe = Pipeline(steps=[('std_slc', std_slc),
                           ('pca', pca),
                           ('dec_tree', dec_tree)])
    
    n_components = list(range(1,X.shape[1]+1,1))
    
    criterion = ['gini', 'entropy']
    max_depth = [2,4,6,8,10,12,14,16]
    
    parameters = dict(pca__n_components=n_components,
                      dec_tree__criterion=criterion,
                      dec_tree__max_depth=max_depth)
    
    clf_GS = GridSearchCV(pipe, parameters)
    clf_GS.fit(X, y)
    
    print('Best Criterion:', clf_GS.best_estimator_.get_params()['dec_tree__criterion'])
    print('Best max_depth:', clf_GS.best_estimator_.get_params()['dec_tree__max_depth'])
    print('Best Number Of Components:', clf_GS.best_estimator_.get_params()['pca__n_components'])
    print(); print(clf_GS.best_estimator_.get_params()['dec_tree'])

In [ ]:
tree_parameters(X_train, y_train)

In [974]:
def find_general_RMSE(total_data):
    RMSE_list = []
    
    for i in range(20):
        train_df, test_df = train_test_split(restocked_data, test_size = 0.25, stratify = restocked_data[['size', 'month', 'days_inbetween']])
        X_train, y_train = forest_variables(train_df)
        X_test, y_test = forest_variables(test_df)
        
        bos_reg = RandomForestRegressor()
        bos_reg.fit(X_train, y_train)
        
        RMSE1 = RMSE(bos_reg, X_test, y_test)
        RMSE_list.append(RMSE1)
        mean = sum(RMSE_list)/len(RMSE_list)
     
    return RMSE_list, mean


    

In [980]:
find_general_RMSE(restocked_data)

C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\20193727\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 t

([5.830951894845301,
  5.744562646538029,
  6.557438524302,
  5.656854249492381,
  5.477225575051661,
  5.744562646538029,
  6.0,
  4.69041575982343,
  5.0990195135927845,
  6.0,
  5.196152422706632,
  4.795831523312719,
  6.0,
  5.0990195135927845,
  5.196152422706632,
  4.898979485566356,
  4.58257569495584,
  5.916079783099616,
  6.244997998398398,
  4.358898943540674],
 5.454485929903163)

In [804]:
y_test

69     10
129    11
181    22
72     12
153    21
34     17
43     18
191    12
174    10
102     7
173    10
8       8
15     13
157    13
186    15
146    14
109    10
110    15
56     10
47     35
134    19
99     28
38     11
91     15
171    11
113     7
71     29
116     9
127     7
4      12
17     12
202    18
196    10
64     17
199     7
57     13
79     14
27     20
140    20
22     13
126    16
148    19
141     7
10     13
84     12
94     17
187    16
46     24
89     20
156    17
166    14
1      14
Name: sales, dtype: int64

In [915]:
for i in range(1, 3):
    print(i)

1
2
